## Data prep

In [ ]:
import os
import time
import zipfile
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import SparseRandomProjection

from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList



###############################################################################
# Data Extraction

# Parameters setup
flakyZip = "Dataset/flaky_files.zip"
nonFlakyZip = "Dataset/nonflaky_files.zip"
extractDir = "extracted"
os.makedirs(extractDir, exist_ok=True)

best_hyperparameter = 'results/best_hyperparameter'
os.makedirs(best_hyperparameter,exist_ok=True)
# Extract the zip files
# Extract and read data once for  non-flaky combination
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

# Print the number of datasets for  combination
print(f"Number of flaky documents ( combination): {len(dataPointsFlaky)}")
print(f"Number of non-flaky documents ( combination): {len(dataPointsNonFlaky)}")
print(f"Total number of documents ( combination): {len(dataPoints)}")

dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))



Number of flaky documents ( combination): 45
Number of non-flaky documents ( combination): 243
Total number of documents ( combination): 288


## KNN

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import json  # Import json to save best hyperparameters
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# KNN with SMOTE and PCA

def runKNNWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()  # Record the start time

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with PCA, SMOTE, and KNN
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('pca', PCA(random_state=42)),                     # PCA for dimensionality reduction
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('knn', KNeighborsClassifier()),                   # KNN classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150, 180, 200, 220],             # PCA components
        'knn__n_neighbors': [3, 5, 7, 9, 11],     # Number of neighbors for KNN
        'knn__weights': ['uniform', 'distance'],          # Weighting options
        'knn__metric': ['euclidean', 'cosine']            # Distance metrics
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Record the end time
    end_time = time.perf_counter()
    total_time = end_time - v0  # Calculate total time taken

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Save the best hyperparameters to a JSON file
    best_params_file = resultsos.path.join(result/best_hyperparameter, f"best_params_knn.json")
    with open(best_params_file, 'w') as f:
        json.dump(best_params, f)
    print(f"Best hyperparameters saved to: {best_params_file}")

    print(f"Best Parameters with SMOTE and PCA: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'n_neighbors': results['params'][idx].get('knn__n_neighbors'),
            'weights': results['params'][idx].get('knn__weights'),
            'metric': results['params'][idx].get('knn__metric'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"knn-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    std_f1 = results['std_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    std_precision = results['std_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    std_recall = results['std_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    std_accuracy = results['std_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]
    std_mcc = results['std_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_mcc})")

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"knn-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_accuracy},{final_precision},{std_precision},{final_recall},{std_recall},{final_f1},{std_f1},{final_mcc},{std_mcc}\n")

    print(f"KNN analysis completed for {n_splits}-folds with SMOTE and PCA. Results saved to: {outFile_final}")

    return best_params, final_f1, std_f1, final_mcc, std_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run KNN with SMOTE and PCA using 5-fold cross-validation
print("\nStarting KNN analysis with SMOTE and PCA for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, std_f1_5folds, final_mcc_5folds, std_mcc_5folds = runKNNWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for KNN with SMOTE and PCA 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_5folds})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_5folds})")



Starting KNN analysis with SMOTE and PCA for 5-fold cross-validation...
Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best hyperparameters saved to: results\best_params_knn.json
Best Parameters with SMOTE and PCA: {'knn__metric': 'cosine', 'knn__n_neighbors': 11, 'knn__weights': 'distance', 'pca__n_components': 150}
Best F1 Score from cross-validation: 0.5832507361919127
Per-fold metrics saved to: results\knn-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.611111111111111 (Std Dev: 0.0878410461157883)
Final Recall: 0.6 (Std Dev: 0.20608041101101565)
Final Accuracy: 0.8750151240169389 (Std Dev: 0.022768874730524673)
Final F1 Score: 0.5832507361919127 (Std Dev: 0.11588720231390487)
Final MCC: 0.5250008288463672 (Std Dev: 0.12183533363893)
KNN analysis completed for 5-folds with SMOTE and PCA. Results saved to: results\knn-results-5-folds.csv

Best results for KNN with SMOTE and PCA 5-fold cross-validation:
Best Parameters: {'knn__me

## SVM

In [ ]:
from sklearn.svm import SVC


###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# SVM with SMOTE and PCA

def runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer, PCA, SMOTE, and SVM
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('pca', PCA(random_state=42)),                     # Dimensionality reduction
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('svm', SVC(probability=True))                     # SVM classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150, 180, 200, 220],          # PCA components
        'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],            # Regularization parameter
        'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid'] # Kernel types
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Save the best hyperparameters to a JSON file
    best_params_file = resultsos.path.join(result/best_hyperparameter, f"best_params_svm.json")
    with open(best_params_file, 'w') as f:
        json.dump(best_params, f)
    print(f"Best hyperparameters saved to: {best_params_file}") 
    print(f"Best Parameters with SMOTE and PCA: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'C': results['params'][idx].get('svm__C'),
            'kernel': results['params'][idx].get('svm__kernel'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"svm-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    std_f1 = results['std_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    std_precision = results['std_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    std_recall = results['std_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    std_accuracy = results['std_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]
    std_mcc = results['std_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_mcc})")

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"svm-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_accuracy},{final_precision},{std_precision},{final_recall},{std_recall},{final_f1},{std_f1},{final_mcc},{std_mcc}\n")

    print(f"SVM analysis completed for {n_splits}-folds with SMOTE and PCA. Results saved to: {outFile_final}")

    return best_params, final_f1, std_f1, final_mcc, std_mcc

###############################################################################
# Main Execution

outDir = "resultsults"
os.makedirs(outDir, exist_ok=True)

# Run SVM with SMOTE and PCA using 5-fold cross-validation
print("\nStarting SVM analysis with SMOTE and PCA for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, std_f1_5folds, final_mcc_5folds, std_mcc_5folds = runSVMWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for SVM with SMOTE and PCA 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_5folds})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_5folds})")



Starting SVM analysis with SMOTE and PCA for 5-fold cross-validation...
Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best hyperparameters saved to: smote-results\best_params_svm.json
Best Parameters with SMOTE and PCA: {'pca__n_components': 220, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score from cross-validation: 0.6914602683178535
Per-fold metrics saved to: smote-results\svm-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8445238095238097 (Std Dev: 0.09399063976169006)
Final Recall: 0.6222222222222222 (Std Dev: 0.19372884193514103)
Final Accuracy: 0.9201451905626135 (Std Dev: 0.023558538994096483)
Final F1 Score: 0.6914602683178535 (Std Dev: 0.12792567952776687)
Final MCC: 0.6720359085671346 (Std Dev: 0.11175482353648197)
SVM analysis completed for 5-folds with SMOTE and PCA. Results saved to: smote-results\svm-results-5-folds.csv

Best results for SVM with SMOTE and PCA 5-fold cross-validation:
Best Parameters: {'pca__

## XGB

In [10]:
import os
import time
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# XGBoost with SMOTE, PCA, and CountVectorizer

def runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer, PCA, SMOTE, and XGBoost
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),               # Vectorizer
        ('pca', PCA(random_state=42)),                                                 # PCA for dimensionality reduction
        ('smote', SMOTE(random_state=42)),                              # SMOTE for oversampling
        ('xgb', XGBClassifier(eval_metric='logloss', random_state=42))  # XGBoost classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150, 180, 200, 220],        # PCA components
        'xgb__n_estimators': [100, 150, 200],             # Number of boosting rounds
        'xgb__max_depth': [3, 5, 7, 10],                  # Maximum depth of a tree
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],      # Learning rate
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE, PCA, and CountVectorizer: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combination
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'n_estimators': results['params'][idx].get('xgb__n_estimators'),
            'max_depth': results['params'][idx].get('xgb__max_depth'),
            'learning_rate': results['params'][idx].get('xgb__learning_rate'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"xgb-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")
    print(f"Final MCC: {final_mcc}")

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"xgb-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Precision,Recall,F1,MCC\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1},{final_mcc}\n")

    print(f"XGBoost analysis completed for {n_splits}-folds with SMOTE, PCA, and CountVectorizer. Results saved to: {outFile_final}")

    return best_params, final_f1, final_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run XGBoost with SMOTE, PCA, and CountVectorizer using 5-fold cross-validation
print("\nStarting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, final_mcc_5folds = runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...
Fitting 5 folds for each of 192 candidates, totalling 960 fits
Best Parameters with SMOTE, PCA, and CountVectorizer: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 150}
Best F1 Score from cross-validation: 0.6997435897435896
Per-fold metrics saved to: results\xgb-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8714285714285716
Final Recall: 0.6
Final Accuracy: 0.920205686630369
Final F1 Score: 0.6997435897435896
Final MCC: 0.677301901085489
XGBoost analysis completed for 5-folds with SMOTE, PCA, and CountVectorizer. Results saved to: results\xgb-results-5-folds.csv

Best results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:
Best Parameters: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 150}
Best F1 Score: 0.6997435897435896
Final

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import json  # Import json to save best hyperparameters
from xgboost import XGBClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# XGBoost with SMOTE, PCA, and CountVectorizer

def runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer, PCA, SMOTE, and XGBoost
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),               # Vectorizer
        ('pca', PCA(random_state=42)),                                  # PCA for dimensionality reduction
        ('smote', SMOTE(random_state=42)),                              # SMOTE for oversampling
        ('xgb', XGBClassifier(
            eval_metric='logloss', 
            use_label_encoder=False,  # Avoids warning about label encoder
            random_state=42))                                           # XGBoost classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150, 180, 200, 220],        # PCA components
        'xgb__n_estimators': [100, 150, 200],             # Number of boosting rounds
        'xgb__max_depth': [3, 5, 7, 10],                  # Maximum depth of a tree
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],      # Learning rate
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Save the best hyperparameters to a JSON file
    best_params_file = resultsos.path.join(result/best_hyperparameter, f"best_params_xgb.json")
    with open(best_params_file, 'w') as f:
        json.dump(best_params, f)
    print(f"Best hyperparameters saved to: {best_params_file}")

    print(f"Best Parameters with SMOTE, PCA, and CountVectorizer: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'n_estimators': results['params'][idx].get('xgb__n_estimators'),
            'max_depth': results['params'][idx].get('xgb__max_depth'),
            'learning_rate': results['params'][idx].get('xgb__learning_rate'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"xgb-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    std_f1 = results['std_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    std_precision = results['std_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    std_recall = results['std_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    std_accuracy = results['std_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]
    std_mcc = results['std_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages with standard deviations)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_mcc})")

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"xgb-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_accuracy},{final_precision},{std_precision},{final_recall},{std_recall},{final_f1},{std_f1},{final_mcc},{std_mcc}\n")

    print(f"XGBoost analysis completed for {n_splits}-folds with SMOTE, PCA, and CountVectorizer. Results saved to: {outFile_final}")

    return best_params, final_f1, std_f1, final_mcc, std_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "resultsults"
os.makedirs(outDir, exist_ok=True)

# Run XGBoost with SMOTE, PCA, and CountVectorizer using 5-fold cross-validation
print("\nStarting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, std_f1_5folds, final_mcc_5folds, std_mcc_5folds = runXGBWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_5folds})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_5folds})")



Starting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...
Fitting 5 folds for each of 192 candidates, totalling 960 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:57:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:57:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:57:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are

Best hyperparameters saved to: smote-results\best_params_xgb.json
Best Parameters with SMOTE, PCA, and CountVectorizer: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 150}
Best F1 Score from cross-validation: 0.6997435897435896
Per-fold metrics saved to: smote-results\xgb-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8714285714285716 (Std Dev: 0.12380952380952381)
Final Recall: 0.6 (Std Dev: 0.08888888888888888)
Final Accuracy: 0.920205686630369 (Std Dev: 0.01742706812304978)
Final F1 Score: 0.6997435897435896 (Std Dev: 0.06615384615384616)
Final MCC: 0.677301901085489 (Std Dev: 0.06862832843938978)
XGBoost analysis completed for 5-folds with SMOTE, PCA, and CountVectorizer. Results saved to: smote-results\xgb-results-5-folds.csv

Best results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:
Best Parameters: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_d

c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:12:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


## Random Forest

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import json  # Import json to save best hyperparameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# Random Forest with SMOTE

def runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer, SMOTE, and Random Forest
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('rf', RandomForestClassifier(random_state=42))    # Random Forest classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'rf__n_estimators': [50, 100, 200],                 # Number of trees in the forest
        'rf__max_depth': [10, 20, 30],                      # Maximum depth of the tree
        'rf__min_samples_split': [5, 10],                   # Minimum number of samples required to split a node
        'rf__min_samples_leaf': [2, 5],                     # Minimum number of samples required at a leaf node
        'rf__criterion': ['gini', 'entropy'],               # Function to measure the quality of a split
        'rf__class_weight': ['balanced']                    # Class weights
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Save the best hyperparameters to a JSON file
    best_params_file = resultsos.path.join(result/best_hyperparameter, f"best_params_rf.json")
    with open(best_params_file, 'w') as f:
        json.dump(best_params, f)
    print(f"Best hyperparameters saved to: {best_params_file}")

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_estimators': results['params'][idx].get('rf__n_estimators'),
            'max_depth': results['params'][idx].get('rf__max_depth'),
            'min_samples_split': results['params'][idx].get('rf__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('rf__min_samples_leaf'),
            'criterion': results['params'][idx].get('rf__criterion'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"rf-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    std_f1 = results['std_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    std_precision = results['std_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    std_recall = results['std_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    std_accuracy = results['std_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]
    std_mcc = results['std_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_mcc})")

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"rf-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_accuracy},{final_precision},{std_precision},{final_recall},{std_recall},{final_f1},{std_f1},{final_mcc},{std_mcc}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile_final}")

    return best_params, final_f1, std_f1, final_mcc, std_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "resultsults"
os.makedirs(outDir, exist_ok=True)

# Run Random Forest with SMOTE using 5-fold cross-validation
print("\nStarting Random Forest analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, std_f1_5folds, final_mcc_5folds, std_mcc_5folds = runRFWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Random Forest with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_5folds})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_5folds})")



Starting Random Forest analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best hyperparameters saved to: smote-results\best_params_rf.json
Best Parameters with SMOTE: {'rf__class_weight': 'balanced', 'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
Best F1 Score from cross-validation: 0.822015823873409
Per-fold metrics saved to: smote-results\rf-smote-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8238888888888889 (Std Dev: 0.048291808471334935)
Final Recall: 0.8222222222222222 (Std Dev: 0.05443310539518171)
Final Accuracy: 0.9444041137326075 (Std Dev: 0.013121372569903846)
Final F1 Score: 0.822015823873409 (Std Dev: 0.04193302466637172)
Final MCC: 0.7898138504372108 (Std Dev: 0.04984274546216977)
Random Forest analysis completed for 5-folds with SMOTE. Results saved to: smote-results\rf-results-5-folds.csv

Best re

## Decision Tree

In [15]:
import os
import time
import numpy as np
import pandas as pd
import json  # Import json to save best hyperparameters
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# Decision Tree with SMOTE

def runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer
    }

    # Define a pipeline with CountVectorizer, SMOTE, and Decision Tree
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Vectorizer
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('dt', DecisionTreeClassifier(random_state=42))    # Decision Tree classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'dt__max_depth': [10, 20, 30],                     # Maximum depth of the tree
        'dt__min_samples_split': [5, 10],                  # Minimum number of samples required to split a node
        'dt__min_samples_leaf': [2, 5],                    # Minimum number of samples required at a leaf node
        'dt__criterion': ['gini', 'entropy'],              # Function to measure the quality of a split
        'dt__max_features': [None, 'sqrt', 'log2'],        # Controls how many features to consider for splits
        'dt__class_weight': ['balanced']                   # Class weights
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    # Save the best hyperparameters to a JSON file
    best_params_file = resultsos.path.join(result/best_hyperparameter, f"best_params_dt.json")
    with open(best_params_file, 'w') as f:
        json.dump(best_params, f)
    print(f"Best hyperparameters saved to: {best_params_file}")

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'max_depth': results['params'][idx].get('dt__max_depth'),
            'min_samples_split': results['params'][idx].get('dt__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('dt__min_samples_leaf'),
            'criterion': results['params'][idx].get('dt__criterion'),
            'max_features': results['params'][idx].get('dt__max_features'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'std_accuracy': results.get('std_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'std_precision': results.get('std_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'std_recall': results.get('std_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'std_f1': results.get('std_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],
            'std_mcc': results.get('std_test_mcc', [None])[idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"dt-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    std_f1 = results['std_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    std_precision = results['std_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    std_recall = results['std_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    std_accuracy = results['std_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]
    std_mcc = results['std_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision} (Std Dev: {std_precision})")
    print(f"Final Recall: {final_recall} (Std Dev: {std_recall})")
    print(f"Final Accuracy: {final_accuracy} (Std Dev: {std_accuracy})")
    print(f"Final F1 Score: {final_f1} (Std Dev: {std_f1})")
    print(f"Final MCC: {final_mcc} (Std Dev: {std_mcc})")

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"dt-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Std_Accuracy,Precision,Std_Precision,Recall,Std_Recall,F1,Std_F1,MCC,Std_MCC\n")
        f.write(f"{final_accuracy},{std_accuracy},{final_precision},{std_precision},{final_recall},{std_recall},{final_f1},{std_f1},{final_mcc},{std_mcc}\n")

    print(f"Decision Tree analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile_final}")

    return best_params, final_f1, std_f1, final_mcc, std_mcc

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "results"
os.makedirs(outDir, exist_ok=True)

# Run Decision Tree with SMOTE using 5-fold cross-validation
print("\nStarting Decision Tree analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, std_f1_5folds, final_mcc_5folds, std_mcc_5folds = runDTWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Decision Tree with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds} (Std Dev: {std_f1_5folds})")
print(f"Final MCC: {final_mcc_5folds} (Std Dev: {std_mcc_5folds})")



Starting Decision Tree analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best hyperparameters saved to: results\best_params_dt.json
Best Parameters with SMOTE: {'dt__class_weight': 'balanced', 'dt__criterion': 'gini', 'dt__max_depth': 10, 'dt__max_features': None, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 5}
Best F1 Score from cross-validation: 0.8495475113122172
Per-fold metrics saved to: results\dt-smote-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.9199999999999999 (Std Dev: 0.16)
Final Recall: 0.8444444444444443 (Std Dev: 0.20608041101101565)
Final Accuracy: 0.955111917725348 (Std Dev: 0.04162577882117278)
Final F1 Score: 0.8495475113122172 (Std Dev: 0.14433963596047805)
Final MCC: 0.8454298941250029 (Std Dev: 0.1399454710554278)
Decision Tree analysis completed for 5-folds with SMOTE. Results saved to: results\dt-results-5-folds.csv

Best results for Decision Tree with SMOTE 5-